In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import math
import folium
import random

uploaded_data = pd.read_excel(r"input\Airtel ZM_2G_Database - 04.01.2024.xlsx")


In [2]:
AZIMUTH = 'Azimuth'
BCCH = 'BCCH'
LAT = 'Lat'
LONG = 'Long'
SECTOR_ID = 'Sector ID'
SITE_ID = 'Site ID'

uploaded_data[AZIMUTH] = pd.to_numeric(uploaded_data[AZIMUTH], errors='coerce')

# Replace NaN values (non-numeric) with 0
uploaded_data[AZIMUTH].fillna(0, inplace=True)

# Convert the entire column to integer type
uploaded_data[AZIMUTH] = uploaded_data[AZIMUTH].astype(int)
uploaded_data[LAT] = pd.to_numeric(uploaded_data[LAT], errors='coerce')

df = uploaded_data.copy()

In [3]:
# AZIMUTH = 'Azimuth'
# BCCH = 'BCCH'
# LAT = 'Lat'
# LONG = 'Long'
# SECTOR_ID = 'Sector ID'
# SITE_ID = 'Site ID'

# AZIMUTH = 'AZIMUTH'
# BCCH = 'BCCH'
# LAT = 'Lat(in decimal)'
# LONG = 'Long(in decimal)'
# SECTOR_ID = 'Cell ID'
# SITE_ID = 'Site ID'

In [3]:
# Convert the column to numeric with errors='coerce' to replace non-numeric values with NaN
# uploaded_data[AZIMUTH] = pd.to_numeric(uploaded_data[AZIMUTH], errors='coerce')

# # Replace NaN values (non-numeric) with 0
# uploaded_data[AZIMUTH].fillna(0, inplace=True)

# # Convert the entire column to integer type
# uploaded_data[AZIMUTH] = uploaded_data[AZIMUTH].astype(int)
# uploaded_data[LAT] = pd.to_numeric(uploaded_data[LAT], errors='coerce')

# Your NBR plan generation code here
cols = df.columns.tolist()

mmfc = df.copy()
plan = df.copy()
# plan = pd.DataFrame()
final = pd.DataFrame(columns=cols)
result = pd.DataFrame()

# plan = pd.concat([plan, df[df['NBR Plan'] != 'Yes']])

df3 = pd.DataFrame()

for Cell_id, Site_id, long_a, lat_a, azi_a in zip(plan[SECTOR_ID], plan[SITE_ID], plan[LONG], plan[LAT], plan[AZIMUTH]):
    Dist = []

    for long_b, lat_b in zip(mmfc[LONG], mmfc[LAT]):
        d = 108 * (math.sqrt((long_a - long_b) ** 2 + (lat_a - lat_b) ** 2))
        Dist.append(d)

    mmfc["Distance"] = Dist
    Dist = []

    mmfc = mmfc.sort_values(by='Distance')

    StoS = []
    StoS_final = []
    StoA = []
    StoA_final = []
    StoB = []
    StoB_final = []
    Azi = []
    Grade = []

    for long_b, lat_b, azi_b in zip(mmfc[LONG], mmfc[LAT], mmfc[AZIMUTH]):


        n = math.degrees(math.atan2(lat_b - lat_a, long_b - long_a))
        StoS.append(n)

        m = -90 - int(n) if n <= -90 else 270 - int(n)
        StoS_final.append(m)

        p = 360 + (int(azi_b) - m) if m > int(azi_b) else int(azi_b) - m
        StoA.append(p)

        q = 360 - p if p > 180 else p
        StoA_final.append(q)

        r = 360 + (azi_a - m) if m > azi_a else azi_a - m
        StoB.append(r)

        s = r - 180 if r > 180 else 180 - r
        StoB_final.append(s)

        t = 10 if (s + q) < 10 else s + q
        Azi.append(t)

        x = d * t
        Grade.append(x)

    mmfc["S to S"] = StoS
    mmfc["S to S Final"] = StoS_final
    mmfc["S to A"] = StoA
    mmfc["S to A Final"] = StoA_final
    mmfc["S to B"] = StoB
    mmfc["S to B Final"] = StoB_final
    mmfc["Azi"] = Azi
    mmfc["Grade"] = Grade

    StoS = []
    StoS_final = []
    StoA = []
    StoA_final = []
    StoB = []
    StoB_final = []
    Azi = []
    Grade = []

    mmfc = mmfc.sort_values (by ='Grade')

    df2 = pd.DataFrame()  # AZIMUTH,  LAT, LONG, SECTOR_ID, SITE_ID
    df2 = mmfc[[SECTOR_ID, SITE_ID, 'Distance', 'Azi', 'Grade']].copy().iloc[:25]
    df2["Cell ID(Plan)"] = Cell_id
    df2["Site ID(Plan)"] = Site_id

    df3 = pd.concat([df3, df2])

selected_columns = ['Cell ID(Plan)', 'Site ID(Plan)', SECTOR_ID, SITE_ID, 'Distance', 'Azi', 'Grade']
df4 = df3[selected_columns]

# Define a list to store the NBR relations
nbr_relations = []

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    lat_a, lon_a = float(row[LAT]), float(row[LONG])
    azimuth_a = float(row[AZIMUTH])

    for _, neighbor_row in df.iterrows():
        lat_b, lon_b = float(neighbor_row[LAT]), float(neighbor_row[LONG])

        # Calculate the distance between cell A and cell B
        distance = 108 * (math.sqrt((lon_a - lon_b) ** 2 + (lat_a - lat_b) ** 2))

        # Calculate the azimuth difference between cell A and cell B
        azimuth_diff = abs(azimuth_a - neighbor_row[AZIMUTH])

        Grade = abs(distance * azimuth_diff)

        # Check if cell B is a neighbor of cell A based on your criteria
        # You can adjust the criteria as needed
        if distance < 10 and azimuth_diff < 30:
            # Append the NBR relation as a dictionary
            nbr_relations.append({
                'Cell A ID': row[SECTOR_ID],
                'Cell B ID': neighbor_row[SECTOR_ID],
                'Distance': distance,
                'Azimuth Difference': azimuth_diff,
                'Grade': Grade
            })

# Convert the list of NBR relations to a DataFrame
nbr_relations_df = pd.DataFrame(nbr_relations)

# # Provide the option to download the NBR relations with the same name as the input file
# nbr_filename = os.path.join(output_folder, f"{remove_extension(uploaded_filename)}_NBR_Relations.xlsx")
# nbr_relations_df.to_excel(nbr_filename, index=False)

# # Convert the NBR relations DataFrame to an HTML table
# nbr_download_link = "/download_nbr_relations"
# nbr_content_df = nbr_relations_df.head(50).reset_index(drop=True)
# nbr_content = nbr_content_df.to_html(classes='table table-striped', escape=False, index=False)

KeyboardInterrupt: 

In [ ]:
nbr_relations_df